<div style="text-align: center;"><br>
<img src="https://assets-global.website-files.com/62b9d45fb3f64842a96c9686/62d84db4aeb2f6552f3a2f78_Quantinuum%20Logo__horizontal%20blue.svg" width="200" height="200" /></div>

# Circuit Stitching

Circuit stitching is a workflow to "stitch" multiple jobs into 1 job using the Mid-Circuit Measurement & Reset (MCMR) feature. The first benefit is to minimize H-System Quantum Credits (HQCs) consumption for experiments consisting of multiple jobs. The second benefit is the reduction of job runtime, because many jobs are stitched into one job submission. For each new job, the H-Series performs a sequence of actions in repetition, which contribute to the overall job runtime. However, by "stitching" many jobs into one job, these actions are reduced. Circuit stitching for a 5-qubit classical shadows use-case, shows a 100x improvement in HQC credit cost. Similarly, a 2.5x improvement is demonstrated for a GHZ State fidelity use-case, as shown in the plot below.

<div style="text-align: center;">
    <img src="figures/circuit_stitching_cost_data.png" width="1000" />
</div>

Stitching involves appending circuits depth-wise and width-wise (across the qubit register). There are two components to cost jobs in terms of HQCs:

1. a +5 contribution for each job submitted to H-Series;
2. a job-specific cost accounting for the number of gate operations and shots.

\begin{equation}
HQC = 5 + \frac{C}{5000} \left( N_{1q} + 10 N_{2q} + 5 N_m \right)
\end{equation}

<div style="text-align: center;">
    <img src="figures/circuit_stitching_figure_1.png" width="1000" />
</div>

The schematic above shows a basic stitching workflow. Instead of submitting multiple jobs to the H-Series device, the number of jobs can be minimized by stitching circuits together depth-wise. If the number of qubits needed is smaller than number of device qubits, then circuits can be stitched in parallel, across the qubit register. To ensure optimal performance, the following guidelines must be followed: 

* TKET programs of size ~4 MB;
* less then 10,000 2-qubit gate operations;
* a total cost of less then 30 HQCs;
* less than a specifc number of classical registers, defined on a per-device basis;
* contain classical registers with a width below a device-specific threshold.

**Contents:**
* [Backend Information](#backend-info)
* [Circuit Stitching Workflow](#circuit-stitching)
* [Classical Shadows](#classical-shadows)

## Backend Information

An instance of the `QuantinuumBackend` is contructed and authenticated.

In [5]:
from pytket.extensions.quantinuum import QuantinuumBackend

backend = QuantinuumBackend(device_name="H1-1E")
backend.login()

This instance contains a `BackendInfo` attribute that reports useful device properties, such as maximum number of qubits, maximum number of classical bit registers and the maximum allowed width of any classical bit register. The `BackendInfo` instance can be exported to a dictionary and printed.

The `get` method can be used directly on the `BackendInfo` instance to obtain the relevant quantities. The code cell below prints the number of qubits, number of classical registers, the max width of any classical register, in addition to device specific information.

In [6]:
backend_info =  backend.backend_info

device_qubits = backend_info.n_nodes
cl_reg_width = backend_info.get_misc("cl_reg_width")
n_cl_reg = backend_info.n_cl_reg
print(f"Number of device qubits:\t{device_qubits}")
print(f"Number of classical registers:\t{cl_reg_width}")
print(f"Max classical register width:\t{n_cl_reg}")
print(f"Supports MCMR:\t\t\t{backend_info.supports_midcircuit_measurement}")
print(f"Supports Fast Feed Forward:\t{backend_info.supports_fast_feedforward}")
print(f"Supports Reset:\t\t\t{backend_info.supports_reset}")

Number of device qubits:	20
Number of classical registers:	32
Max classical register width:	120
Supports MCMR:			True
Supports Fast Feed Forward:	True
Supports Reset:			True


## Circuit Stitching

### GHZ State Preparation

The $N$-qubit GHZ state preparation, using the definition,

$$\begin{equation}
| {GHZ}_N \rangle = \frac{1}{\sqrt{2}} \left( {| 0 \rangle}^{\bigotimes N} + {| 1 \rangle}^{\bigotimes N} \right),
\end{equation}$$

where $N=5$.

The `circuit_stitching` method requires a list of circuits as input, in addition to a BackendInfo instance. The method will parallelize the circuits across the qubit register (stitch width-wise) and then stitch the circuits depth-wise. The `BackendInfo` instance informs the method on how many circuits can be parallelzsed. The method assumes that all circuits inputted as a list have the same number of qubits.

In [7]:
from pytket.circuit import Circuit

n_qubits = 5
circ = Circuit(n_qubits)
circ.H(0)
for i in range(n_qubits-1):
    circ.CX(i,i+1)

The functions below generate the measurement circuits necessary to measure the GHZ state fidelity.

* `generate_population_circuit`: Generates a $N$-qubit circuit measuring the GHZ state in the computational basis.
* `generate_parity_circuits`: Generates $N$-sets of $N$-qubit circuits, each with all qubits measuring the GHZ state in the $X$-$Y$ plane.

In [8]:
from typing import Tuple, List
from pytket.circuit import Circuit

def generate_population_circuit(
    ghz_circuit: Circuit,
) -> Circuit:
    circuit = ghz_circuit.copy()
    circuit.add_barrier(circuit.qubits)
    circuit.measure_all()
    return circuit

def generate_parity_circuits(
    ghz_circuit: Circuit,
) -> Tuple[List[Circuit], List[float]]:
    circuits = []
    angles = []
    for k in range(1, ghz_circuit.n_qubits + 1):
        circuit = ghz_circuit.copy()
        angle = k / ghz_circuit.n_qubits
        for q in circuit.qubits:
            circuit.Rz(-angle, q)
            circuit.Ry(-0.5, q)
        circuit.measure_all()
        circuits += [circuit]
        angles += [angle]
    return circuits, angles

In [9]:
population_circuit = generate_population_circuit(circ)

parity_circuit_list, angles = generate_parity_circuits(circ)

### Circuit Stitching Setup

The reset operation is required to "stitch" measurement circuits. After each measurement operation, a Reset operation is applied to set the qubit to the $|0 \rangle$ state before proceeding with the operations in the next "stitched" circuit.

In [10]:
from pytket.circuit import Circuit, CircBox, Qubit, BitRegister
from pytket.backends.backendinfo import BackendInfo


def reset_operations(
    n_qubits: int
) -> CircBox:
    r"""Generate a n-qubit CircBox instance containing OpType.Reset operations.

    :param n_qubits: Number of qubits used in the CircBox instance
    :param_type int:
    :returns: CircBox
    """
    circuit = Circuit(n_qubits)
    circuit.name = "Reset"
    for q in circuit.qubits:
        circuit.Reset(q)
    return CircBox(circuit)

The `circuit_stitching` method accepts a list of circuits and attempts to stitch in parallel, across the qubit register, and then secondly depth-wise. The `input_circuit` list must be consistent in the number of qubits. This routine is followed until all circuits have been stitched. The number of classical registers on the output circuit is equal to the number of circuits stitched together. For parallel stitching, the function first checks the number of qubits in the circuit to be stitched are less than the number of device qubits. If this is true, a parameter, ``p`, estimates the number of times the input circuits will fit into the register. Based on this, a list of list of circuits is constructed. Each sublist is stitched depth-wise. Each circuit in the sublist is stitched in parallel across the register.

<div style="text-align: center;">
    <img src="figures/stitching_workflow.png" width="1200" />
</div>

In [13]:
from typing import List, Dict, Tuple

import numpy as np

from pytket.circuit import Circuit, CircBox, Qubit, BitRegister
from pytket.backends.backendinfo import BackendInfo

def _add_circuit(
    circuit_instance: Circuit,
    circbox: CircBox,
    index: int,
    qubits: List[Qubit],
    resetbox: CircBox = None
):
    creg = circuit_instance.add_c_register(f"creg_{index}", len(qubits))
    circuit_instance.add_circbox(circbox, qubits + list(creg))
    if resetbox:
        circuit_instance.add_circbox(resetbox, qubits)


def circuit_stitching(
    input_circuits: List[Circuit], 
    backend_info: BackendInfo,
) -> Tuple[Circuit, Dict[int, BitRegister]]:
    r"""Generate a stitched circuit based on a list of input circuits. The circuit is 
    stitched width-wise and then depth-wise.

    :param input_circuit: Circuit instance to stitch `n` times.
    :param backend_info: BackendInfo instance containing information on number 
        of device qubits, number of allowed classical register and maximum width 
        for each classical register.
    :returns: Circuit
    """
    if all(input_circuits[0].n_qubits != c.n_qubits for c in input_circuits[1:]):
        raise ValueError("All circuits should have the same number of qubits.")
    n_qubits_device = backend_info.architecture.nodes
    p = np.floor_divide(len(n_qubits_device), input_circuits[0].n_qubits)
    num_c_reg = 0
    circuit = Circuit(len(n_qubits_device))
    reset_box = reset_operations(input_circuits[0].n_qubits)

    circuits_list = []
    for i in range(0, len(input_circuits), p):
        circuits_list += [input_circuits[i:i+p]]

    for k, circuits in enumerate(circuits_list):
        for i, c in enumerate(circuits):
            circ = c.copy()
            circ.name = f"box_{num_c_reg}"
            a = i * circ.n_qubits
            b = (i + 1) * circ.n_qubits
            qubits = circuit.qubits[a: b]
            if k == len(circuits_list) - 1:
                reset_box = None
            _add_circuit(circuit, CircBox(circ), num_c_reg, qubits, resetbox=reset_box)
            num_c_reg += 1
    return circuit

The `circuit_stitching method` is used to stitch the 5-qubit GHZ population circuit and 5 GHZ parity circuits.

In [14]:
input_circuits = [population_circuit] + parity_circuit_list
stitched_circuit = circuit_stitching(input_circuits, backend.backend_info)

The resulting circuit contains 6 distinct classical registers. Each classical register corresponds to a subcircuit in the stitched circuit. Note, it is necessary to order the register based on the register name suffix (an integer). This ensures the order of the register is the same as the ordering of the stitched circuits.

In [15]:
registers = stitched_circuit.c_registers
registers.sort(key=lambda a: int(a.name.split("_")[-1]))

In [18]:
print(registers)

[BitRegister("creg_0", 5), BitRegister("creg_1", 5), BitRegister("creg_2", 5), BitRegister("creg_3", 5), BitRegister("creg_4", 5), BitRegister("creg_5", 5)]


The classical registers are required to post-process the result after H-Series execution is complete. The classical register information should be saved to `JSON`, enabling this information to be used across python sessions. The ordering of the classical registers needs to preserved in `JSON` format, and the a record should be kept of the name and size of each classical register.

In [19]:
import json

register_data = {reg.name: reg.size for reg in registers}

with open("stitched_registers.json", "w") as json_io:
    json.dump(register_data, json_io)

### Costing Analysis

The stitched circuit is compiled to use the native H-Series gateset. The individual circuits, used to construct the stitched circuit, are also compiled for the costing analysis.

In [20]:
compiled_stitched_circuit = backend.get_compiled_circuit(stitched_circuit, optimisation_level=2)
compiled_circuits = backend.get_compiled_circuits([population_circuit] + parity_circuit_list, optimisation_level=2)

Then all circuits will be submitted and costed with 100 shots each.

In [21]:
n_shots = 100

The function below calculates the HQC cost of a circuit, based on the number of operations and number of shots. As mentioned previously, there is a +5 overhead for each new job submitted to H-Series for execution.

In [22]:
from pytket.circuit import Circuit, OpType
from pytket.backends.backendinfo import BackendInfo


def _cost(
    circuit: Circuit,
    n_shots: int
) -> int:
    r"""Local estimation of HQCs (H-System Quantum Credits) 
    needed to execute stitched circuit.

    :param circuit: circuit instance.
    :returns: int
    """
    n1q = circuit.n_1qb_gates()
    n2q = circuit.n_2qb_gates()
    nm = circuit.n_gates_of_type(OpType.Measure)
    return n_shots/5000 * (n1q + 10 * n2q + 5 * nm) + 5

Each measurement circuit approximately requires 40 HQCs, but the stitched circuit requires only 15 HQCs. By stitching the measurement circuits, an 2.5x decrease is observed in the required number of HQCs. 

In [23]:
sum(_cost(c, n_shots) for c in compiled_circuits)

39.32000000000001

In [24]:
_cost(compiled_stitched_circuit, n_shots)

14.32

The function below verifies the stitched circuit satisfies the various performance constraints before submission to H-Series:

* less then 10,000 2-qubit gate operations;
* a total cost of less then 30 HQCs;
* less than a specific number of classical registers, defined on a per-device basis;
* contain classical registers with a width below a device-specific threshold.

In [25]:
def verify_stitched_circuit(
    circuit: Circuit,
    n_shots: int,
    backend_info: BackendInfo
) -> Tuple[bool, Dict[str, float | int]]:
    """Verify the stitched circuit satisfies basic 
    constraints before submission to H-Series.

    :param circuit: `Circuit` instance to stitch `n` times.
    :param backend_info: `BackendInfo` instance containing information on number 
        of device qubits, number of allowed classical register and maximum width 
        for each classical register.
    :returns: bool
    """
    check = True
    data = {}
    data["cost"] = [_cost(circuit, n_shots), 30]
    data["n_2qb_gates"] = [circuit.n_2qb_gates(), 10000]
    data["c_registers"] = [len(circuit.c_registers), backend_info.n_cl_reg]
    data["max_cl_reg_width"] = [max([a.size for a in circuit.c_registers]), backend_info.get_misc("cl_reg_width")]

    if _cost(circuit, n_shots) > 30:
        check = False
    if circuit.n_2qb_gates() > 10000:
        check = False
    if len(circuit.c_registers) > backend_info.n_cl_reg:
        check = False
    if all(a.size > backend_info.get_misc("cl_reg_width") for a in circuit.c_registers):
        check = False
        
    return check, data

The method returns a boolean and a dictionary. The boolean determines whether the stitched circuit satisfies the performance constraints before submission (True is success). The dictionary reports the cost, two-qubit gate count, number of classical registers and maximum size across all classical registers. In addition, the limit for each property is reported.

In [26]:
check, data = verify_stitched_circuit(compiled_stitched_circuit, 100, backend.backend_info)
print(check)
print(data)

True
{'cost': [14.32, 30], 'n_2qb_gates': [24, 10000], 'c_registers': [6, 120], 'max_cl_reg_width': [5, 32]}


`QuantinuumBackend` can be used to cost the stitched circuit as well. This will be similar to the estimate above.

In [ ]:
cost_hqc = backend.cost(compiled_stitched_circuit, n_shots=100, syntax_checker="H1-1SC")
print(cost_hqc)

`QuantinuumBackend` is used to submit the circuit for execution to H-Series with 100 shots. This is equivalent to measuring each individual circuit with 100 shots each.

In [ ]:
handle = backend.process_circuit(compiled_stitched_circuit, n_shots=100)

### Result Destitching

`QuantinuumBackend` is used to check the status of the stitched circuit job.

In [ ]:
backend.circuit_status(handle)

CircuitStatus(status=<StatusEnum.COMPLETED: 'Circuit has completed. Results are ready.'>, message='{"name": "job", "submit-date": "2024-08-22T17:18:28.109559", "result-date": "2024-08-22T17:18:45.585046", "queue-position": null, "cost": "17.78", "error": null}', error_detail=None, completed_time=None, queued_time=None, submitted_time=None, running_time=None, cancelled_time=None, error_time=None, queue_position=None)

`QuantinuumBackend` can be used to retrieve the result from the backend.

In [ ]:
result = backend.get_result(handle)

The register JSON is opended and loaded into this jupyter session.

In [ ]:
import json

with open("stitched_registers.json", "r") as json_io:
    register_data = json.load(json_io)

The JSON data is used to instiate a list of `BitRegister` instances.

In [ ]:
from pytket.circuit import BitRegister

register_list = [BitRegister(name, size) for name, size in register_data.items()]

The result for the stitched circuit needs to be *destitched* into effective measurement results for each individual circuit.

In [ ]:
from pytket.backends.backendresult import BackendResult
from pytket.utils.outcomearray import OutcomeArray

destitched_results = []
for reg in register_list:
    outcome_array = OutcomeArray.from_readouts(result.get_shots(reg))
    destitched_results += [BackendResult(shots=outcome_array)]

The dictionaries below report the effective measurement result for each individual measurement circuit. The effective measurement result is matches the ordering of the measurement circuits as inputted into the `circuit_stitching` function.

In [ ]:
for r in destitched_results:
    print(r.get_distribution())

{(0, 0, 0, 0, 0): 0.47, (1, 1, 1, 1, 1): 0.53}
{(0, 0, 0, 0, 0): 0.1, (0, 0, 0, 0, 1): 0.01, (0, 0, 0, 1, 1): 0.05, (0, 0, 1, 0, 1): 0.07, (0, 0, 1, 1, 0): 0.07, (0, 1, 0, 0, 1): 0.05, (0, 1, 0, 1, 0): 0.07, (0, 1, 1, 0, 0): 0.04, (0, 1, 1, 1, 1): 0.06, (1, 0, 0, 0, 1): 0.06, (1, 0, 0, 1, 0): 0.04, (1, 0, 1, 0, 0): 0.05, (1, 0, 1, 1, 1): 0.04, (1, 1, 0, 0, 0): 0.07, (1, 1, 0, 1, 1): 0.06, (1, 1, 1, 0, 1): 0.06, (1, 1, 1, 1, 0): 0.1}
{(0, 0, 0, 0, 1): 0.09, (0, 0, 0, 1, 0): 0.04, (0, 0, 1, 0, 0): 0.02, (0, 0, 1, 1, 1): 0.05, (0, 1, 0, 0, 0): 0.1, (0, 1, 0, 1, 0): 0.01, (0, 1, 0, 1, 1): 0.08, (0, 1, 1, 0, 1): 0.07, (0, 1, 1, 1, 0): 0.08, (1, 0, 0, 0, 0): 0.05, (1, 0, 0, 1, 1): 0.06, (1, 0, 1, 0, 0): 0.01, (1, 0, 1, 0, 1): 0.03, (1, 0, 1, 1, 0): 0.03, (1, 1, 0, 0, 1): 0.03, (1, 1, 0, 1, 0): 0.05, (1, 1, 1, 0, 0): 0.06, (1, 1, 1, 1, 0): 0.01, (1, 1, 1, 1, 1): 0.13}
{(0, 0, 0, 0, 0): 0.01, (0, 0, 0, 0, 1): 0.04, (0, 0, 0, 1, 0): 0.1, (0, 0, 1, 0, 0): 0.09, (0, 0, 1, 1, 1): 0.06, (0, 1, 0, 0

## Classical Shadows

Estimating properties of unknown quantum states is a key objective of quantum computing. Classical shadow approximation is one method to estimate those properties [[Huang et al](https://pennylane.ai/qml/demos/tutorial_classical_shadows/#huang2020)]. The workflow presented below reconstructs the classical shadow representation of an unknown quantum state. Initially, an $n$-qubit quantum state is prepared by a circuit, and renamed unitary $\hat{U}$ is applied,

$$\begin{equation}
    \rho \rightarrow \hat{U} \rho \hat{U}^{\dagger}.
\end{equation}$$

Next, we measure in the computational basis and obtain a bit string of outcomes $|b \rangle = |0011...10\rangle$. If the unitaries, $\hat{U}$, are chosen at random from a particular ensemble, then we can store the reverse operation, $\hat{U} | b \rangle \langle b | \hat{U}^{\dagger}$, efficiently in classical memory. We call this a snapshot of the state. Moreover, we can view the average over these snapshots as a measurement channel,

$$\begin{equation}
E \left[\hat{U} | b \rangle \langle b | \hat{U}^{\dagger} \right] = M(\rho).
\end{equation}$$

If the ensemble of unitaries defines a tomographically complete set of measurements, we can invert the channel and reconstruct the state,

$$\begin{equation}
\rho = E \left[ M^{-1} \left( \hat{U} | b \rangle \langle b | \hat{U}^{\dagger} \right) \right]
\end{equation}$$

If we apply the procedure outlined above $N$-times, then the collection of inverted snapshots is what we call the classical shadow,

$$\begin{equation}
    S(\rho, N) = \left\{ \hat{\rho}_1 = M^{-1} \left( \hat{U}_1 | b_1 \rangle \langle b_1 | \hat{U}^{\dagger}_1 \right), ... , \hat{\rho}_N = M^{-1} \left( \hat{U}_N | b_N \rangle \langle b_N | \hat{U}^{\dagger}_N \right) \right\}.
\end{equation}$$

The code-cell below generates subciruit containing measurement operations on qubits. This function requires a [`pytket.pauli.QubitPauliString`](https://tket.quantinuum.com/api-docs/pauli.html#pytket.pauli.QubitPauliString) as input and based on the [`pytket.pauli.Pauli`](https://tket.quantinuum.com/api-docs/pauli.html#pytket.pauli.Pauli) operation on each qubit, performs the relevant basis transformation on the circuit.

In [ ]:
from typing import Tuple, List

from numpy.random import choice

from pytket.circuit import Circuit, Bit, Qubit
from pytket.partition import MeasurementSetup, MeasurementBitMap
from pytket.pauli import Pauli, QubitPauliString

def create_measurement_basis(
    qps: QubitPauliString, 
    n_qubits: int
) -> Circuit:
    circuit = Circuit(n_qubits)
    for qubit, pauli in qps.map.items():
        bit = Bit(qubit.index[0])
        circuit.add_bit(bit)
        if pauli == Pauli.X:
            circuit.H(qubit).Measure(qubit, bit)
        elif pauli == Pauli.Y:
            circuit.V(qubit).Measure(qubit, bit)
        elif pauli == Pauli.Z:
            circuit.Measure(qubit, bit)
        else:
            continue
    return circuit

The code-cell below generates $N$ subcircuits containing [`OpType.Measure`](https://tket.quantinuum.com/api-docs/optype.html#pytket.circuit.OpType) operations and basis transformations gates. The number of subcircuits is determined by the input parameter `measurement_budget`. This function calls `create_measurement_basis` internally. The number of qubits, `n_qubits` also needs to be passed as an integer. `n_qubits` must be equal to the number of qubits on the state-preparation circuit.

In [ ]:
def construct_randomized_shadows(
    measurement_budget: int,
    n_qubits: int,
) -> Tuple[List[Circuit], MeasurementSetup]:
    measurement_setup = MeasurementSetup()
    circuit_list = []
    PAULIS = [Pauli.X, Pauli.Y, Pauli.Z]
    for j in range(measurement_budget):
        qps = QubitPauliString({Qubit(i): choice(PAULIS) for i in range(n_qubits)})
        measurement_circ = create_measurement_basis(qps, n_qubits)
        circuit_list += [measurement_circ]
        measurement_setup.add_measurement_circuit(measurement_circ)
        if qps == QubitPauliString():
            continue
        bits = []
        qps_items = qps.map.items()
        if len(qps_items) != len(qps.map):
            continue
        for qubit, _ in qps_items:
            bits += [qubit.index[0]]
        if len(bits) > 0:
            bits.sort()
            bitmap = MeasurementBitMap(j, bits)
            measurement_setup.add_result_for_term(qps, bitmap)
    return circuit_list, measurement_setup

The code-cell below generates 100 5-qubit subcircuits. 

In [ ]:
measurement_budget = 100
n_qubits = 5

mcircs, ms = construct_randomized_shadows(measurement_budget, n_qubits)

The code-cell below synthesises the 5-qubit GHZ state.

In [ ]:
from pytket.circuit import Circuit

circ = Circuit(n_qubits)
circ.H(0)
for i in range(n_qubits-1):
    circ.CX(i,i+1)

The code-cell below generates each measurement circuit by appending the measurement subcircuit to a copy of the state-preparation circuit. The circuit is compiled and submitted to H-Series with 1-shot.

In [ ]:
handles = []
for i, mc in enumerate(mcircs):
    c = circ.copy()
    c.name = f"job-{i}"
    c.append(mc)
    cc = backend.get_compiled_circuit(c, optimisation_level=2)
    h = backend.process_circuit(cc, n_shots=1)
    handles += [h]

The results handles are saved to disk and in the subsequent cell, reloaded to disk. This is an important step if using `ResultHandle` instances across python sessions.

In [ ]:
import json

with open("result_handles_4.json", "w") as json_io:
    json.dump([str(h) for h in handles], json_io)

In [ ]:
import json

with open("result_handles_4.json", "r") as json_io:
    handles_data = json.load(json_io)

In [ ]:
from pytket.backends.resulthandle import ResultHandle

handles = [ResultHandle.from_str(h) for h in handles_data]

The `ResultHandle` can be used to retrieve results.

In [ ]:
results = backend.get_results(handles)

The code-cell below defines a function to reconstruct the state, $\rho$, using the expression below,

$$\begin{equation}
    \hat{\rho} = \bigotimes_{j=1}^{n} \left( 3 \hat{U}_j^{\dagger} | \hat{b}_j \rangle \langle \hat{b}_j | \hat{U}_j - I \right).
\end{equation}$$

The snapshot, $\rho$ would be averaged over multiple rounds.

In [ ]:
import numpy as np

from pytket.backends.backendresult import BackendResult
from pytket.partition import MeasurementSetup

def shadow_reconstruction(
    results: List[BackendResult], 
    measurement_setup: MeasurementSetup
) -> np.ndarray:
    zero_state = np.array([[1, 0], [0, 0]])
    one_state = np.array([[0, 0], [0, 1]])  
    factor = 1 / np.sqrt(2)
    H = factor * np.asarray([[1, 1], [1, -1]])
    vdg = factor * np.asarray([[1, 1j], [1j, 1]], dtype=complex)
    identity = np.eye(2)

    shadow_rho = np.zeros((2 ** n_qubits, 2 ** n_qubits), dtype=complex)
    for qps, bitmap_list in list(measurement_setup.results.items()):
        for bm in bitmap_list:
            r = results[bm.circ_index]
            assert len(bm.bits) == n_qubits
            rho_snapshot = [1]
            for pauli, bit in zip(qps.map.values(), r.get_shots()[0]):
                if bit:
                    state = one_state
                else:
                    state = zero_state
                
                if pauli == Pauli.X:
                    op = H
                elif pauli == Pauli.Y:
                    op = vdg
                else:
                    op = identity
                local_rho = 3 * (op.conj().T @ state @ op) - identity
                rho_snapshot = np.kron(rho_snapshot, local_rho)
            shadow_rho += rho_snapshot
    return shadow_rho / measurement_budget

The density matrix of the GHZ state is reconstructed using classical shadows below.

In [ ]:
shadow_state = shadow_reconstruction(results, ms)

array([[ 4.7187500e-01+0.00000000e+00j, -3.0937500e-02+8.43750000e-03j,
         5.2500000e-02+3.75000000e-02j, ...,
        -1.5187500e-01+2.02500000e-01j, -7.5937500e-02+1.26562500e-01j,
         3.0375000e-01-1.51875000e-01j],
       [-3.0937500e-02-8.43750000e-03j,  8.7500000e-03+0.00000000e+00j,
        -1.8281250e-01+7.03125000e-02j, ...,
        -1.4343750e-01-2.53125000e-02j,  1.5187500e-01-1.51875000e-01j,
         2.2781250e-01-2.53125000e-02j],
       [ 5.2500000e-02-3.75000000e-02j, -1.8281250e-01-7.03125000e-02j,
        -1.1875000e-02+0.00000000e+00j, ...,
         1.5187500e-01-1.77635684e-17j,  8.4375000e-03+1.77187500e-01j,
         4.4408921e-17+5.06250000e-02j],
       ...,
       [-1.5187500e-01-2.02500000e-01j, -1.4343750e-01+2.53125000e-02j,
         1.5187500e-01+1.77635684e-17j, ...,
        -1.9000000e-01+0.00000000e+00j, -1.4906250e-01-6.46875000e-02j,
         1.8750000e-02-1.87500000e-03j],
       [-7.5937500e-02-1.26562500e-01j,  1.5187500e-01+1.51875000e-0

The theoretical GHZ density matrix is printed below.

In [ ]:
state = circ.get_statevector()

ghz_state = np.outer(state, state.conj().T)

array([[0.5+0.j, 0. +0.j, 0. +0.j, ..., 0. +0.j, 0. +0.j, 0.5+0.j],
       [0. +0.j, 0. +0.j, 0. +0.j, ..., 0. +0.j, 0. +0.j, 0. +0.j],
       [0. +0.j, 0. +0.j, 0. +0.j, ..., 0. +0.j, 0. +0.j, 0. +0.j],
       ...,
       [0. +0.j, 0. +0.j, 0. +0.j, ..., 0. +0.j, 0. +0.j, 0. +0.j],
       [0. +0.j, 0. +0.j, 0. +0.j, ..., 0. +0.j, 0. +0.j, 0. +0.j],
       [0.5+0.j, 0. +0.j, 0. +0.j, ..., 0. +0.j, 0. +0.j, 0.5+0.j]])

### Stitched Classical Shadows

The code-cell below estimates the number of circuits that can be stitched together into one job. Since, the measurement budget is 100, all circuits can be stitched into one circuit. The workflow below performs classical shadows but with the circuit stitching workflow. It is important to note that the stitched circuit is submitted with 1-shot. A 100x improvement in HQC cost is demonstrated with circuit stitching compared to regular job submission.

In [ ]:
10000 / circ.n_2qb_gates()

2500.0

In [ ]:
circuits = []
for mc in mcircs:
    c = circ.copy()
    c.append(mc)
    circuits += [c]

In [ ]:
job_circuit = circuit_stitching(circuits, backend.backend_info)

In [ ]:
job_compiled_circuit = backend.get_compiled_circuit(job_circuit, optimisation_level=2)

In [ ]:
handle = backend.process_circuit(job_compiled_circuit, n_shots=1)

In [ ]:
import json

register_data = {reg.name: reg.size for reg in job_compiled_circuit.c_registers}

with open("stitched_registers_shadows.json", "w") as json_io:
    json.dump(register_data, json_io)

In [ ]:
result = backend.get_result(handle)

In [ ]:
import json
from pytket.circuit import BitRegister

with open("stitched_registers_shadows.json", "r") as json_io:
    register_data = json.load(json_io)

register_list = [BitRegister(name, size) for name, size in register_data.items()]
register_list.sort(key=lambda k: int(k.name.split("_")[-1]))

In [ ]:
from pytket.utils.outcomearray import OutcomeArray

destitched_results = []
for reg in register_list:
    outcome_array = OutcomeArray.from_readouts(result.get_shots(reg))
    destitched_results += [BackendResult(shots=outcome_array)]

In [ ]:
stitched_shadow_state = shadow_reconstruction(destitched_results, ms).real

array([[ 0.455    , -0.0271875, -0.12375  , ..., -0.354375 ,  0.0253125,
         0.30375  ],
       [-0.0271875, -0.053125 , -0.1321875, ..., -0.0084375,  0.151875 ,
        -0.2784375],
       [-0.12375  , -0.1321875, -0.00625  , ...,  0.151875 ,  0.1434375,
         0.10125  ],
       ...,
       [-0.354375 , -0.0084375,  0.151875 , ..., -0.195625 , -0.0984375,
        -0.14625  ],
       [ 0.0253125,  0.151875 ,  0.1434375, ..., -0.0984375, -0.0025   ,
         0.0234375],
       [ 0.30375  , -0.2784375,  0.10125  , ..., -0.14625  ,  0.0234375,
         0.704375 ]])

The code-cell below calculates the 2-norm between the reconstucted GHZ state using circuit stitching and the state not using circuit stitching.

In [ ]:
def operator_2_norm(R):
    return np.sqrt(np.trace(R.conjugate().transpose() @ R))

operator_2_norm(stitched_shadow_state - shadow_state.real)

<div align="center"> &copy; 2024 by Quantinuum. All Rights Reserved. </div>